In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from platypus import NSGAII, Problem, Real

## Initialize reservoirs

In [2]:
from Reservoir4 import Reservoir

ice_harbor = Reservoir(SA=9200*4047,capacity=603000,tail_elev=339,pool_elev=446,bottom_elev=310,fish_pass=0.965, pc=106_000, spillway_cap=850_000, alfa=2.2, beta=4.9)
lower_monumental = Reservoir(SA=6590*4047,capacity=810000,tail_elev=439,pool_elev=548.3,bottom_elev=406,fish_pass=0.965, pc=130_000, spillway_cap=850_000, alfa=2.2, beta=4.9)
little_goose = Reservoir(SA=10025*4047,capacity=903000,tail_elev=539,pool_elev=646.5,bottom_elev=500,fish_pass=0.9775, pc=130_000, spillway_cap=850_000, alfa=2.2, beta=4.9)
lower_granite = Reservoir(SA=8900*4047,capacity=810000,tail_elev=636,pool_elev=746.5,bottom_elev=590,fish_pass=1, pc=130_000, spillway_cap=850_000, alfa=2.2, beta=4.9)

## Import data

In [3]:
# we only need lower granite inflow and tributary flow for each reservoir. Importing outflow data for comparison.
pre_cutoff = datetime(1970,1,1)
post_cutoff = datetime(1993,1,1)

# lower granite
lower_granite_data = pd.read_csv('lowergraniteinflow.csv')
lower_granite_outflows = pd.read_csv('lowergraniteoutflow.csv')
lower_granite_data = lower_granite_data.merge(lower_granite_outflows, how='inner', on=['date'])
lower_granite_data['date'] = pd.to_datetime(lower_granite_data['date'])
lower_granite_data['L (unit:cfs)'] = 0 #NO TRIB DATA FOR LOWER GRANITE
lower_granite_pre = lower_granite_data[lower_granite_data['date'] <= pre_cutoff]
lower_granite_post = lower_granite_data[lower_granite_data['date'] >= post_cutoff]

# little goose
little_goose_data = pd.read_csv('littlegooseoutflow.csv')               
little_goose_data['L (unit:cfs)'] = 0 #NO TRIB DATA FOR LITTLE GOOSE
little_goose_data['date'] = pd.to_datetime(little_goose_data['date'])
little_goose_pre = little_goose_data[little_goose_data['date'] <= pre_cutoff]
little_goose_post = little_goose_data[little_goose_data['date'] >= post_cutoff]

# lower monumental
lower_monumental_trib = pd.read_csv('lowermontrib.csv')
lower_monumental_outflows = pd.read_csv('lowermonumentaloutflow.csv')
lower_monumental_trib['L (unit:cfs)'] = lower_monumental_trib['L (unit:cfs)'].clip(lower=0)
lower_monumental_data = lower_monumental_trib.merge(lower_monumental_outflows, how='inner', on=['date'])
lower_monumental_data['date'] = pd.to_datetime(lower_monumental_data['date'])
lower_monumental_pre = lower_monumental_data[lower_monumental_data['date'] <= pre_cutoff]
lower_monumental_post = lower_monumental_data[lower_monumental_data['date'] >= post_cutoff]

# ice harbor
ice_harbor_outflow = pd.read_csv('iceharboroutflow.csv')
ice_harbor_trib = pd.read_csv('iceharbortrib.csv')
ice_harbor_outflows = pd.read_csv('iceharboroutflow.csv')
ice_harbor_trib['L (unit:cfs)'] = ice_harbor_trib['L (unit:cfs)'].clip(lower=0)
ice_harbor_data = ice_harbor_trib.merge(ice_harbor_outflows, how='inner', on=['date'])
ice_harbor_data['date'] = pd.to_datetime(ice_harbor_data['date'])
ice_harbor_pre = ice_harbor_data[ice_harbor_data['date'] <= pre_cutoff]
ice_harbor_post = ice_harbor_data[ice_harbor_data['date'] >= post_cutoff]

## Simulate reservoirs in series

In [8]:
# lower granite simulation post 1993
keep_LGR=1
initial_storage_LGR = 670*0.3046
param_LGR = {'mef':50000*(0.3046**3), 'h1':710*0.3046, 'm':1000}
datetimes=lower_granite_post['date']
prev_out_LGR=lower_granite_post['H (unit:cfs)'].values*0.3046**3
tributary_LGR=lower_granite_post['L (unit:cfs)'].values*0.3046**3
    
LGR_outflow, LGR_hydro = lower_granite.simulate(keep_LGR, initial_storage_LGR, param_LGR, datetime=datetimes, prev_out=prev_out_LGR, tributary=tributary_LGR)

# little goose simulation
keep_LGS=1
initial_storage_LGS = 800*0.3046
param_LGS = {'mef':60000*(0.3046**3), 'h1':760*0.3046, 'm':1000}
datetimes=little_goose_post['date']
prev_out_LGS=LGR_outflow
tributary_LGS=little_goose_post['L (unit:cfs)'].values*0.3046**3

LGS_outflow, LGS_hydro = little_goose.simulate(keep_LGS, initial_storage_LGS, param_LGS, datetime=datetimes, prev_out=prev_out_LGS, tributary=tributary_LGS)

# lower monumental simulation
keep_LMN=1
initial_storage_LMN = 900*0.3046
param_LMN = {'mef':70000*(0.3046**3), 'h1':810*0.3046, 'm':1000}
datetimes=lower_monumental_post['date']
prev_out_LMN=LGS_outflow
tributary_LMN=lower_monumental_post['L (unit:cfs)'].values*0.3046**3

LMN_outflow, LMN_hydro = lower_monumental.simulate(keep_LMN, initial_storage_LMN, param_LMN, datetime=datetimes, prev_out=prev_out_LMN, tributary=tributary_LMN)

# ice harbor simulation
keep_ICH=1
initial_storage_ICH = 500*0.3046
param_ICH = {'mef':80000*(0.3046**3), 'h1':600*0.3046, 'm':1000}
datetimes=ice_harbor_post['date']
prev_out_ICH=LMN_outflow
tributary_ICH=ice_harbor_post['L (unit:cfs)'].values*0.3046**3  

ICH_outflow, ICH_hydro = ice_harbor.simulate(keep_ICH, initial_storage_ICH, param_ICH, datetime=datetimes, prev_out=prev_out_ICH, tributary=tributary_ICH)